# Base de dades MySQL
**Descripció**  
Crear base de dades amb MySQL.

## Nivell 1
### Exercici 1
Crea una base de dades relacionals senzilla utilitzant MySQL(https://www.mysql.com/) i connecta-la a Python

## Nivell 2
### Exercici 2
Mostra que pots carregar algunes consultes senzilles a un Pandas Dataframe.

## Nivell 3
### Exercici 3
Genera algun gràfic que resumeixi les dades. 